# Airline Incidents Safety Data Jupyter Notebook

Data is from https://www.kaggle.com/datasets/tarique7/airline-incidents-safety-data

Last Update: May 4 2023

In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

import copy

import autokeras as ak

import tensorflow as tf

2023-05-04 22:25:36.566825: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Step 0: Test GPU

In [ ]:
print(f'\nTensorflow version = {tf.__version__}\n')
print(f'\n{tf.config.list_physical_devices("GPU")}\n')

In [ ]:
import torch

print(f'\nAvailable cuda = {torch.cuda.is_available()}')

print(f'\nGPUs availables = {torch.cuda.device_count()}')

print(f'\nCurrent device = {torch.cuda.current_device()}')

print(f'\nCurrent Device location = {torch.cuda.device(0)}')

print(f'\nName of the device = {torch.cuda.get_device_name(0)}')

## Step 1: Read dataset

In [3]:
data = pd.read_csv('data/Airline Occurences.csv')

## Step 2: Understanding the dataset

In [4]:
data.head()

,Report,Part Failure,Occurence Nature condition,Occurence Precautionary Procedures
0,MECHANICAL / LANDING GEAR GND FAIL MSG AFTER T...,RT MLG BRAKE DAMAGED,WARNING INDICATION,OTHER
1,THE NOSE LANDING GEAR DID NOT EXTEND FULLY DUR...,ZONE 700 MALFUNCTIONED,WARNING INDICATION,ABORTED APPROACH
2,THE LEFT SIDE HYDRAULIC SYSTEM FILTER BOWL ASS...,HYD FILTER FAILED,OTHER,ABORTED APPROACH
3,AIRCRAFT WAS ON ROLLOUT DURING A NORMAL LANDIN...,LEFT COLLAPSED,OTHER,OTHER
4,UPON TAKEOFF ROLL BUT PRIOR TO REACHING 80 KNO...,ZONE 600 CRACKED,WARNING INDICATION,ABORTED TAKEOFF


In [5]:
data.shape

(100028, 4)

In [6]:
data.dtypes

Report                                object
Part Failure                          object
Occurence Nature condition            object
Occurence Precautionary Procedures    object
dtype: object

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100028 entries, 0 to 100027
Data columns (total 4 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Report                              100028 non-null  object
 1   Part Failure                        100028 non-null  object
 2   Occurence Nature condition          100028 non-null  object
 3   Occurence Precautionary Procedures  100028 non-null  object
dtypes: object(4)
memory usage: 3.1+ MB


In [8]:
print(data.isnull().sum())

Report                                0
Part Failure                          0
Occurence Nature condition            0
Occurence Precautionary Procedures    0
dtype: int64


In [9]:
data['Occurence Nature condition'].value_counts()

OTHER                         82172
WARNING INDICATION             7154
NO TEST                        3577
SMOKE/FUMES/ODORS/SPARKS       2166
FALSE WARNING                  1683
FLUID LOSS                      975
FLT CONT AFFECTED               320
OVER TEMP                       316
NO WARNING INDICATION           255
VIBRATION/BUFFET                228
INADEQUATE Q C                  223
AFFECT SYSTEMS                  202
F.O.D.                          191
PARTIAL RPM/PWR LOSS            152
MULTIPLE FAILURE                107
FLAME/FIRE                       65
ENGINE STOPPAGE                  56
SIGNIFICANT FAILURE REPORT       53
INFLIGHT SEPARATION              47
ENGINE FLAMEOUT                  38
ELECT. POWER LOSS-50 PC          27
FLT. ATTITUDE INST.              21
Name: Occurence Nature condition, dtype: int64

In [10]:
data['Occurence Precautionary Procedures'].value_counts()

NONE                        79268
OTHER                       14855
UNSCHED LANDING              3627
ABORTED TAKEOFF               719
RETURN TO BLOCK               556
EMER. DESCENT                 422
ENGINE SHUTDOWN               229
ABORTED APPROACH              142
DEACTIVATE SYST/CIRCUITS      135
DUMP FUEL                      25
ACTIVATE FIRE EXT.             22
O2 MASK DEPLOYED               21
AUTOROTATION                    4
CABIN DEPRESSURIZATION          3
Name: Occurence Precautionary Procedures, dtype: int64

## Step 3: Text Preprocessing

In [11]:
def text_preprocessing(data):
    stop_words = set(stopwords.words('english'))
    data_columns_list = list(data.columns)
    for i in range(len(data_columns_list)):
        data[data_columns_list[i]] = data[data_columns_list[i]].str.strip().str.lower()
        for j in range(len(data)):
            text = data[data_columns_list[i]].loc[j]
            text = ' '.join([word for word in text.split() if word not in stop_words])
            data[data_columns_list[i]].loc[j] = text
    return data

In [12]:
list(data.columns)

['Report',
 'Part Failure',
 'Occurence Nature condition',
 'Occurence Precautionary Procedures']

In [13]:
data_processed = copy.deepcopy(data) # Remove reference pointers
data_output = text_preprocessing(data_processed)

In [14]:
data_output.head()

,Report,Part Failure,Occurence Nature condition,Occurence Precautionary Procedures
0,mechanical / landing gear gnd fail msg takeoff...,rt mlg brake damaged,warning indication,
1,nose landing gear extend fully approach seen i...,zone 700 malfunctioned,warning indication,aborted approach
2,left side hydraulic system filter bowl assembl...,hyd filter failed,,aborted approach
3,aircraft rollout normal landing. landing gear ...,left collapsed,,
4,upon takeoff roll prior reaching 80 knots pilo...,zone 600 cracked,warning indication,aborted takeoff


## Step 4: Model Building

In [15]:
X = data_output[['Report', 'Part Failure']]
X_new = data_output['Report']
y = data_output['Occurence Nature condition']
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.25, random_state=42)

In [16]:
X_train_ak = np.array(X_train)
X_test_ak = np.array(X_test)
y_train_ak = np.array(y_train)
y_test_ak = np.array(y_test)

In [17]:
print(X_train_ak.shape)
print(X_test_ak.shape)
print(y_train_ak.shape)
print(y_test_ak.shape)

(75021,)
(25007,)
(75021,)
(25007,)


In [18]:
keras = ak.TextRegressor(overwrite=True, max_trials=1)

keras.fit(X_train_ak, y_train_ak, epochs=30, validation_split=0.2)

Trial 1 Complete [00h 00m 04s]

Best val_loss So Far: None
Total elapsed time: 00h 00m 04s
INFO:tensorflow:Oracle triggered exit
Epoch 1/30


2023-05-04 22:27:29.479242: W tensorflow/core/framework/op_kernel.cc:1757] OP_REQUIRES failed at cast_op.cc:121 : UNIMPLEMENTED: Cast string to float is not supported


UnimplementedError: Graph execution error:

Detected at node 'mean_squared_error/Cast' defined at (most recent call last):
    File "/Users/kelvincykong/anaconda3/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/kelvincykong/anaconda3/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/Users/kelvincykong/anaconda3/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/kelvincykong/anaconda3/lib/python3.10/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/Users/kelvincykong/anaconda3/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 711, in start
      self.io_loop.start()
    File "/Users/kelvincykong/anaconda3/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/Users/ke

In [ ]:
keras_export = keras.export_model()
keras_export.summary()